#### 继承Module类来构造模型

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden=nn.Linear(784,256)
        self.act=nn.ReLU()
        self.output=nn.Linear(256,10)
    def forward(self,X):
        a=self.act(self.hidden(X))
        return self.output(a)

net(X)会调用MLP继承自Module类的__call__函数，这个函数将调用MLP类定义的forward函数来完成前向计算。

In [2]:
X=torch.rand(2,784)
net=MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.3061, -0.1070, -0.0387,  0.0655, -0.1268,  0.0184,  0.0231,  0.0412,
         -0.0342, -0.0209],
        [ 0.2492, -0.1104, -0.0763, -0.0103, -0.1123, -0.0116, -0.0697,  0.0193,
         -0.0856, -0.0136]], grad_fn=<AddmmBackward>)

#### Module的子类

Module类是一个通用的部件。Pytorch还实现了继承自Module的可以方便构建模型的类:如Sequential、ModuleList和ModuleDict等等.

##### Sequential类

Sequential是简单的串联网络，该模型的前向计算就是将输入按添加的顺序逐一计算。

##### ModuleList类

ModuleList接受一个子模块的列表作为输入，然后也可以类似LIst那样进行append和extend操作.

ModuleList和Sequential的区别:ModuleList仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序(所以不用保证邻层的输入输出维度匹配),而且没有实现forward功能;而Sequential内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配，内部forward功能已经实现.

ModuleList和python里面的List的区别是，加入到ModuleList里面的所有模块的参数会被自动添加到整个网络中。

In [3]:
net=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)
#net(torch.zeros(1,784)) #会报错

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


##### ModuleDict类

ModuleDict接收一个子模块的字典作为输入，然后也可以类似字典那样进行添加访问操作。

和ModuleList一样,ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数。同样,ModuleDict也与Python的Dict有所不同,ModuleDict里的所有模块的参数会被自动添加到整个网络中。

In [5]:
net=nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU(),
})
net['output']=nn.Linear(256,10)
print(net['output'])
print(net.output)
print(net)

Linear(in_features=256, out_features=10, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


##### 构造复杂的模型

上面介绍的子类可以使模型构造更加简单，但直接继承Module类可以极大地扩展模型构造的灵活性。